# Tesla specific time series processing
The goal of this notebook is to demonstrate the implementation of time series processing steps that are specific to Tesla.

## Setup

### Imports

In [ ]:
import plotly.express as px

from pandas.api.types import CategoricalDtype

from core.pandas_utils import *
from transform.processed_tss.config import IN_CHARGE_CHARGING_STATUS_VALS, IN_DISCHARGE_CHARGING_STATUS_VALS
from transform.processed_tss.ProcessedTimeSeries import TeslaProcessedTimeSeries
from transform.raw_results.tesla_results import get_results

### Data extraction

In [ ]:
! mkdir -p data_cache

In [ ]:
tss = TeslaProcessedTimeSeries()

In [ ]:
tss = tss.astype({
    "vin": CategoricalDtype(),
    "charging_status": CategoricalDtype(),
    "charging_method": CategoricalDtype(),
})

## Segmentation and indexing

In [ ]:
def compute_charge_mask(tss:DF) -> DF:
    base = (
        Series(pd.NA, index=tss.index, dtype="boolean")
        .mask(tss["charging_status"].isin(IN_CHARGE_CHARGING_STATUS_VALS), True)
        .mask(tss["charging_status"].isin(IN_DISCHARGE_CHARGING_STATUS_VALS), False)
    )
    ffill_base = base.groupby(tss["vin"], observed=False).ffill()
    bfill_base = base.groupby(tss["vin"], observed=False).bfill()
    base = base.mask(ffill_base.eq(bfill_base), ffill_base)
    base = base.mask(tss["soc"] >= 98)
    tss["in_charge"] = base
    return tss

def compute_charge_idx(tss:DF) -> DF:
    tss_grp = tss.groupby("vin", observed=False)
    tss["charge_energy_added"] = tss_grp["charge_energy_added"].ffill()
    power_loss = tss_grp['charge_energy_added'].diff().div(tss["sec_time_diff"])
    # min_power_loss = (
    #     power_loss
    #     .loc[tss["charging_status"] == 'stopped']
    #     .quantile(0.05)
    # )
    min_power_loss = 0.0001
    new_charge_mask = power_loss.lt(min_power_loss, fill_value=0) | tss["time_diff"].gt(TD(days=1))
    tss["in_charge_idx"] = new_charge_mask.groupby(tss["vin"], observed=False).cumsum()
    return tss

def compute_status_col(tss:DF) -> DF:
    tss_grp = tss.groupby("vin", observed=False)
    status = tss["in_charge"].map({True: "charging", False:"discharging", pd.NA:"unknown"})
    tss["status"] = status.mask(
        tss["in_charge"].eq(False, fill_value=True),
        np.where(tss_grp["odometer"].diff() > 0, "moving", "idle_discharging"),
    )
    return tss

In [ ]:
tss = (
    tss
    .pipe(compute_charge_mask)
    .pipe(compute_charge_idx)
    .pipe(compute_status_col)
)

Visualization of vin that had the most amount of charges in previous implementation.

In [ ]:
TARGET_VIN = "LRW3E7FA4MC314534"
px.scatter(
    tss.query("vin == @TARGET_VIN"),
    x="date",
    y="soc",
    color="status",
    hover_data=["odometer"]
)

Thibault's vehicle.

In [ ]:
THIBAULT_VIN = "5YJ3E7EB7KF474436"
px.scatter(
    tss.query("vin == @THIBAULT_VIN"),
    x="date",
    y="soc",
    color="status",
    symbol="charging_status",
    hover_data=["odometer", "charging_status"]
).update_layout(showlegend=True)

Visualization of masking with 4 random vins.

In [ ]:
vin_samples = tss["vin"].pipe(uniques_as_series).sample(n=4)
px.scatter(
    tss.query("vin in @vin_samples"),
    x="date",
    y="soc",
    color="status",
    symbol="in_charge_idx",
    facet_row="vin",
    hover_data=["odometer", "charging_status"],
    height=750,
).update_yaxes(matches=None)

Usually, very long charging periods are caused by some edge case scenario that prevents our pipeline to correctly segment/separate multiple periods.

In [ ]:
charge_lengths = (
    tss
    .query("status == 'charging'")
    .groupby(["vin", "in_charge_idx"], observed=True)
    .agg(start_date=pd.NamedAgg("date", "first"), end_date=pd.NamedAgg("date", "last"))
    .eval("duration = end_date - start_date")
    .sort_values(by="duration")
)


In [ ]:
LONG_CHARGES_VINS = charge_lengths.index.get_level_values(0)[-7:]
px.scatter(
    tss.query("vin == '5YJSA7E59PF494292'"),
    x="date",
    y="soc",
    color="status",
    symbol="in_charge_idx",
    facet_row="vin",
    hover_data=["odometer"],
    height=750,
).update_yaxes(matches=None)